In [4]:
import pandas as pd
pd.set_option('display.max_rows', None)
import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib import path
import os
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy
import numpy as np
import netCDF4 as nc
np.set_printoptions(threshold=100000)
from shapely.geometry import Polygon, Point, MultiPoint
from shapely.ops import cascaded_union, unary_union, transform
from datetime import datetime, timedelta
import datetime
import math
from scipy.ndimage.interpolation import shift
import shapely.wkt
from shapely.validation import explain_validity,make_valid
import xarray as xr
import pygeos as pg
import time
import seaborn as sns
from my_functions import sat_vap_press, vap_press, hot_dry_windy, haines

In [ ]:
#all fires
#fire_incidents = ['BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH', 'WILLIAMS FLATS', 'SHADY','PEDRO MOUNTAIN', 'WALKER', '204 COW']

#2020 fires
#fire_incidents = ['AUGUST COMPLEX','BOBCAT', 'DOLAN', 'HOLIDAY FARM','CREEK', 'LAKE', 'CAMERON PEAK', 'PINE GULCH']
fire_incidents = ['LAKE']
#fire_incidents=['CAMERON PEAK']
path_poly = '/data2/lthapa/ML_daily/fire_polygons/'

start_time=12
for jj in range(len(fire_incidents)):
    print(fire_incidents[jj])
    fire_daily = gpd.read_file(path_poly+fire_incidents[jj].lower().replace(' ', '_')+'_VIIRS_daily_'+str(start_time)+'Z_day_start.geojson') #polygons and attributes
    
    
    #get rid of rows/cols we don't need
    fire_daily=fire_daily.drop(columns=['Current Overpass'])
    fire_daily = fire_daily.drop(np.where(fire_daily['geometry']==None)[0])
    fire_daily['fire area (ha)'] = fire_daily['geometry'].area/10000 #hectares
    fire_daily.set_geometry(col='geometry', inplace=True) #designate the geometry column
    fire_daily = fire_daily.rename(columns={'Current Day':'UTC Day', 'Local Day': str(start_time)+ 'Z Start Day'})
    
       
    fire_daily = fire_daily.iloc[np.array(fire_daily['UTC Day'].values,dtype='datetime64')<=np.datetime64('2020-08-19'),:]
    #print(fire_daily)
    
    
    
    #merra
    #me = merra_timeseries(fire_daily, start_time)
    #print(me)
    #me.to_csv('./fire_features/'+fire_incidents[jj].lower().replace(' ', '_')+'_Daily_MERRA_Moving_Average_2_'+str(start_time)+'Z.csv') #daily averages

    #rave
    #rave = rave_timeseries(fire_daily, times,start_time)
    #print(rave)
    #rave.to_csv('./fire_features/'+fire_incidents[jj].lower().replace(' ', '_')+'_Daily_RAVE_'+str(start_time)+'Z.csv')
    
    #esi
    #esi = esi_timeseries(fire_daily, times)
    #print(esi)
    #esi.to_csv('./fire_features/'+fire_incidents[jj].lower().replace(' ', '_')+'_Daily_ESI.csv')
    
    #pws
    #pws = pws_timeseries(fire_daily, times)
    #print(pws)
    #pws.to_csv('./fire_features/'+fire_incidents[jj].lower().replace(' ', '_')+'_Daily_PWS.csv')
    
    #imerg
    #imerg = imerg_fwi_timeseries(fire_daily, times)
    #print(imerg)
    #imerg.to_csv('./fire_features/'+fire_incidents[jj].lower().replace(' ', '_')+'_Daily_FWI_IMERG.csv')
    
    #cpc
    #cpc = cpc_fwi_timeseries(fire_daily,times)
    #print(cpc)
    #cpc.to_csv('./fire_features/'+fire_incidents[jj].lower().replace(' ', '_')+'_Daily_FWI_CPC.csv')
    
    #hrrr
    #hrrr = hrrr_timeseries(fire_daily,start_time)

In [ ]:
times_end = np.minimum(np.datetime64(fire_daily['UTC Day'].iloc[len(fire_daily)-1])+np.timedelta64(1,'D'), 
                      np.datetime64('2020-10-31'))
    times = pd.date_range(np.datetime64(fire_daily['UTC Day'].iloc[0]),\
                         times_end)

In [ ]:
fire_daily = gpd.read_file('./fire_polygons/august_complex_VIIRS_daily_12Z_day_start.geojson') #polygons and attributes
tic=time.time()
fire_daily = fire_daily.to_crs(epsg=4326)
toc=time.time()
print(fire_daily)
print(toc-tic)

#tic=time.time()
#fire_daily_total = fire_daily.dissolve(by='Fire Name')
#toc=time.time()
#print(fire_daily_total)
#print(toc-tic)

#print(fire_daily['geometry'].iloc[0])
#hrrr_grid = gpd.read_file('HRRR_GRID.geojson',rows=10)
#print(hrrr_grid)
#tic = time.time()


In [ ]:
#:
tic= time.time()
for ii in range(1):#len(fire_daily)):
    hrrr_grid = gpd.read_file('HRRR_GRID.geojson',mask=fire_daily['geometry'].iloc[ii])
    
toc=time.time()
#print(hrrr_grid)
print(toc-tic)

In [12]:
fire_daily = gpd.read_file('./fire_polygons/lake_VIIRS_daily_12Z_day_start.geojson') #polygons and attributes
fire_daily = fire_daily.to_crs(epsg=4326)
print(fire_daily['geometry'].iloc[0].bounds)

tic= time.time()
hrrr_grid = gpd.read_file('HRRR_GRID.geojson',bbox=fire_daily['geometry'].iloc[0].bounds)
toc=time.time()
print(hrrr_grid)
print(toc-tic)

(-118.57547009085407, 34.63046041345746, -118.45018028129323, 34.71835637672822)
          lat         lon  row  col  \
0   34.619087 -118.490150  456  261   
1   34.625168 -118.458282  456  262   
2   34.633114 -118.561287  457  259   
3   34.639214 -118.529419  457  260   
4   34.645306 -118.497551  457  261   
5   34.651390 -118.465675  457  262   
6   34.657463 -118.433792  457  263   
7   34.659328 -118.568710  458  259   
8   34.665432 -118.536835  458  260   
9   34.671528 -118.504951  458  261   
10  34.677612 -118.473068  458  262   
11  34.683689 -118.441177  458  263   
12  34.685547 -118.576134  459  259   
13  34.691654 -118.544250  459  260   
14  34.697750 -118.512360  459  261   
15  34.703838 -118.480461  459  262   
16  34.709915 -118.448563  459  263   
17  34.711765 -118.583572  460  259   
18  34.717873 -118.551674  460  260   
19  34.723972 -118.519775  460  261   
20  34.730061 -118.487869  460  262   

                                             geometry  
0   

/home/lthapa/anaconda3/envs/ML_py/lib/python3.7/site-packages/geopandas/geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


## MERRA

In [ ]:
def merra_timeseries(df,day_start_hour):
    df_merra = pd.DataFrame({'day': np.zeros(len(df)),'temp':np.zeros(len(df)), 'vpd':np.zeros(len(df)), 
                             'wind':np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    #load in the grid
    merra_grid = gpd.read_file('MERRA_GRID.geojson')
    merra_grid = merra_grid.to_crs(epsg=3347) #put into lambert conformal conic 
    
    #do the intersection, not with a for loop!
    fire_merra_intersection = gpd.overlay(df, merra_grid, how='intersection',keep_geom_type=False)
    fire_merra_intersection['grid intersection area (ha)'] =fire_merra_intersection['geometry'].area/10000
    fire_merra_intersection['weights'] = fire_merra_intersection['grid intersection area (ha)']/fire_merra_intersection['fire area (ha)'] 
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_merra_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        #get the time
        df_sub = fire_merra_intersection.iloc[np.where(fire_merra_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
        intersection_sub = df_sub.to_xarray() #polygon and weights for today

        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+np.timedelta64(1,'D'))
        files_back = make_merra_file_namelist(times_back)
        
        #load in all the merra files associated with this lookback window
        dat_merra = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    
        #add the derived data (svp, vp, vpd)
        dat_merra=dat_merra.assign(ESAT=sat_vap_press(dat_merra.TLML))
        dat_merra=dat_merra.assign(E=vap_press(dat_merra.QLML, dat_merra.TLML))
        dat_merra=dat_merra.assign(VPD=dat_merra.ESAT-dat_merra.E)
        
        merra_daily_mean = dat_merra.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        merra_daily_mean_region = merra_daily_mean.sel(lat = np.unique(intersection_sub['lat'].values),
                                  lon = np.unique(intersection_sub['lon'].values)) #get the location of the overlaps
        
        hd0 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((merra_daily_mean_region['SPEEDLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((merra_daily_mean_region['TLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_merra.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',t,hd0,w,hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        dat_merra.close()
        count =count+1
    return df_merra
    

In [ ]:
def make_merra_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/MERRA2/WESTUS_MERRA2_400.inst1_2d_lfo_Nx.FULLDATE.nc4'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
        if (time[jj].strftime('%Y%m')=='202009'):
            base_filename_list[jj] = base_filename_list[jj].replace('400','401')
    return base_filename_list

## RAVE

In [ ]:
#only need to do this once
times_rave_coverage = pd.date_range(np.datetime64('2020-04-07'), np.datetime64('2020-10-31'))
print(times_rave_coverage)
base_filename_nc = '/data2/lthapa/YEAR/AprYEAR_to_OctYEAR/Hourly_Emissions_FV3_13km_FULLDATE0000_FULLDATE2300.nc'
base_filename_xr = '/data2/lthapa/YEAR/AprYEAR_to_OctYEAR/Hourly_Emissions_FV3_13km_FULLDATE0000_FULLDATE2300xr.nc'
for jj in range(len(times_rave_coverage)):
    filename_nc = base_filename_nc.replace('YEAR',times_rave_coverage[jj].strftime('%Y')).\
                                    replace('FULLDATE',times_rave_coverage[jj].strftime('%Y%m%d'))
    filename_xr = base_filename_xr.replace('YEAR',times_rave_coverage[jj].strftime('%Y')).\
                                    replace('FULLDATE',times_rave_coverage[jj].strftime('%Y%m%d'))
    #print(filename_nc, filename_xr)
    
    cmd = 'ncrename -v Latitude,lat -v Longitude,lon ' + filename_nc + ' '+ filename_xr
    print(cmd)
    os.system(cmd)

In [ ]:
def rave_timeseries(df,time, day_start_hour):
    varis = ['Mean_FRP', 'FRE', 'CO2', 'CO', 'SO2', 'OC', 'BC', 'PM2.5', 'NOx', 'NH3'] #don't need 'area', it's the area of each cell
    
    
    #load in the grid
    rave_grid = gpd.read_file('RAVE_GRID.geojson')
    rave_grid = rave_grid.to_crs(epsg=3347)

    #do the intersection, not with a for loop!
    fire_rave_intersection = gpd.overlay(fire_daily, rave_grid, how='intersection',keep_geom_type=False)
    fire_rave_intersection['grid intersection area (ha)'] =fire_rave_intersection['geometry'].area/10000
    fire_rave_intersection['weights'] = fire_rave_intersection['grid intersection area (ha)']/fire_rave_intersection['fire area (ha)'] 
    
    
    fire_rave_intersection = fire_rave_intersection.set_index([str(day_start_hour)+ 'Z Start Day', 'row', 'col'])
    fire_rave_intersection_xr = fire_rave_intersection.to_xarray()
    
    #load in rave data associated with the fire
    time = pd.date_range(np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[0]),
                        np.datetime64(df[str(day_start_hour)+ 'Z Start Day'].iloc[len(df)-1])+
                        np.timedelta64(1,'D'))
    rave_filenames = make_rave_file_namelist(time)
    dat_rave = xr.open_mfdataset(rave_filenames,concat_dim='Time',combine='nested',compat='override', coords='all')
    dat_rave = dat_rave.assign_coords({'Time': dat_rave.time}) #assign coords so we can resample along time
    dat_rave = dat_rave.resample(Time='24H',base=day_start_hour).sum(dim='Time') #take the daily sum

    
    #select the locations and times we want
    dat_rave_sub = dat_rave.isel(yFRP = fire_rave_intersection_xr['row'].values.astype(int), 
                    xFRP = fire_rave_intersection_xr['col'].values.astype(int)).sel(
                    Time = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                         'T'+str(day_start_hour)+':00:00'))
    ndays = len(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'])
    
    #preallocate space for the output
    df_rave = pd.DataFrame({'day':np.zeros(ndays),'Mean_FRP':np.zeros(ndays),\
                          'FRE':np.zeros(ndays),  'CO2':np.zeros(ndays),  'CO':np.zeros(ndays),\
                          'SO2':np.zeros(ndays),  'OC':np.zeros(ndays),  'BC':np.zeros(ndays),\
                          'PM2.5':np.zeros(ndays),  'NOx':np.zeros(ndays),  'NH3':np.zeros(ndays)})

    df_rave['day'].iloc[:] = pd.to_datetime(fire_rave_intersection_xr[str(day_start_hour)+ 'Z Start Day'].values+
                                         'T'+str(day_start_hour)+':00:00')
    for var in varis:
        df_rave[var] = np.nansum(fire_rave_intersection_xr['weights'].values*dat_rave_sub[var].values, axis=(1,2))
    
    return df_rave
    

In [ ]:
def make_rave_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/AprYEAR_to_OctYEAR/Hourly_Emissions_FV3_13km_FULLDATE0000_FULLDATE2300xr.nc'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
    return base_filename_list

## ESI (Evaporative Stress Index)

In [ ]:
def esi_timeseries(df, time):
    #load in the grid
    esi_grid = gpd.read_file('ESI_GRID.geojson')
    esi_grid = esi_grid.to_crs(epsg=3347)

    #do the intersection, not with a for loop!
    fire_esi_intersection = gpd.overlay(fire_daily, esi_grid, how='intersection',keep_geom_type=False)
    fire_esi_intersection['grid intersection area (ha)'] =fire_esi_intersection['geometry'].area/10000
    fire_esi_intersection['weights'] = fire_esi_intersection['grid intersection area (ha)']/fire_esi_intersection['fire area (ha)'] 
    
    inds_esi_range = np.where(pd.to_datetime(fire_esi_intersection['Current Day'].values)<=np.datetime64('2020-10-31'))
    
    fire_esi_intersection = fire_esi_intersection.iloc[inds_esi_range]
    fire_esi_intersection = fire_esi_intersection.set_index(['Current Day', 'lat', 'lon'])
    fire_esi_intersection_xr = fire_esi_intersection.to_xarray()
    
    #load in esi data associated with the fire
    esi_filenames, esi_times = make_esi_file_namelist(times)
    
    #open the esi files
    dat_esi = xr.open_mfdataset(esi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_esi = dat_esi.assign_coords({'time': esi_times}) #assign coords so we can resample along time

    dat_esi_daily = dat_esi.reindex(time=times,method='nearest')
    dat_esi_daily_sub = dat_esi_daily.sel(lat = fire_esi_intersection_xr['lat'].values, 
                                          lon = fire_esi_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_esi_intersection_xr['Current Day'].values), method='nearest')
                                          

    ndays = len(fire_esi_intersection_xr['Current Day'])
    
    #preallocate space for the output
    df_esi = pd.DataFrame({'day':np.zeros(ndays),'ESI':np.zeros(ndays)})

    df_esi['day'].iloc[:] = pd.to_datetime(fire_esi_intersection_xr['Current Day'].values)
    varis=['ESI']
    for var in varis:
        df_esi[var] = np.nansum(fire_esi_intersection_xr['weights'].values*dat_esi_daily_sub['Band1'].values, axis=(1,2))
    
    return df_esi


In [ ]:
def make_esi_file_namelist(time):
    base_filename = '/data2/lthapa/DFPPM_4WK_YEARJDAY.nc'
    
    dates_weekly = pd.date_range(np.datetime64('2020-01-08'), np.datetime64('2020-12-31'),freq='7D')
    fire_start=time[0]
    fire_end=time[len(time)-1]
    
    fire_times_esi = dates_weekly[np.where((dates_weekly>=fire_start)&(dates_weekly<=fire_end))] #weeks in the ESI record where we had fire
    
    base_filename_list = np.repeat(base_filename, len(fire_times_esi))
    
    for jj in range(len(fire_times_esi)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',fire_times_esi[jj].strftime('%Y')).\
                                    replace('JDAY',fire_times_esi[jj].strftime('%j'))
    return base_filename_list, fire_times_esi
    

## PWS (Plant Water Sensitivity)

In [ ]:
def pws_timeseries(df, time_fire):
    #load in the grid
    pws_grid = gpd.read_file('PWS_GRID.geojson')
    pws_grid = pws_grid.to_crs(epsg=3347)

    #do the intersection, not with a for loop!
    fire_pws_intersection = gpd.overlay(fire_daily, pws_grid, how='intersection',keep_geom_type=False)
    fire_pws_intersection['grid intersection area (ha)'] =fire_pws_intersection['geometry'].area/10000
    fire_pws_intersection['weights'] = fire_pws_intersection['grid intersection area (ha)']/fire_pws_intersection['fire area (ha)'] 
    
    inds_pws_range = np.where(pd.to_datetime(fire_pws_intersection['Current Day'].values)<=np.datetime64('2020-10-31'))
    
    fire_pws_intersection = fire_pws_intersection.iloc[inds_pws_range]
    fire_pws_intersection = fire_pws_intersection.set_index(['Current Day', 'lat', 'lon'])
    fire_pws_intersection_xr = fire_pws_intersection.to_xarray()
    
    #load in PWS data associated with the fire (it's only one dataset)
    
    #open the PWS files
    path_pws = '/data2/lthapa/PWS_6_jan_2021.nc'
    dat_pws = xr.open_dataset(path_pws) #map is fixed in time
    dat_pws = dat_pws.assign_coords({'time': time_fire})
    dat_pws_daily = dat_pws['Band1'].expand_dims({'time': time_fire}) #the PWS expanded over all the days
    
    dat_pws_daily_sub = dat_pws_daily.sel(lat = fire_pws_intersection_xr['lat'].values, 
                                          lon = fire_pws_intersection_xr['lon'].values,
                      time = pd.to_datetime(fire_pws_intersection_xr['Current Day'].values), method='nearest')
                                          
    ndays = len(fire_pws_intersection_xr['Current Day'])
    
    #preallocate space for the output
    df_pws = pd.DataFrame({'day':np.zeros(ndays),'PWS':np.zeros(ndays)})

    df_pws['day'].iloc[:] = pd.to_datetime(fire_pws_intersection_xr['Current Day'].values)
    varis=['PWS']
    for var in varis:
        df_pws[var] = np.nansum(fire_pws_intersection_xr['weights'].values*dat_pws_daily_sub.values, axis=(1,2))
    
    return df_pws


In [ ]:
def calculate_grid_cell_corners(LAT, LON):
    #we will assume the very edges of the polygons don't touch the boundary of the domain
    lat_corners = (LAT[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LAT[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    lon_corners = (LON[0:(LAT.shape[0]-1),  0:(LAT.shape[1])-1] + LON[1:(LAT.shape[0]), 1:(LAT.shape[1])])/2
    return lat_corners, lon_corners

In [ ]:
esi_filenames, esi_times = make_esi_file_namelist(times)
#open the esi files
dat_esi = xr.open_mfdataset(esi_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
dat_esi = dat_esi.assign_coords({'time': esi_times}) #assign coords so we can resample along time
dat_esi = dat_esi.reindex(time=times,method='nearest')
print(dat_esi)

In [ ]:
lat_lake = np.array([34.625, 34.675, 34.725])
print(dat_esi['lat'].values)
print(dat_esi.sel(lat=lat_lake, method='nearest'))

## IMERG FWI

In [ ]:
def imerg_fwi_timeseries(df, time):
    varis = ['IMERG.FINAL.v6_DC','IMERG.FINAL.v6_DMC','IMERG.FINAL.v6_FFMC',
             'IMERG.FINAL.v6_ISI','IMERG.FINAL.v6_BUI','IMERG.FINAL.v6_FWI',
             'IMERG.FINAL.v6_DSR'] 

    #load in the grid
    imerg_grid = gpd.read_file('IMERG_FWI_GRID.geojson')
    imerg_grid = imerg_grid.to_crs(epsg=3347)

    #do the intersection, not with a for loop!
    fire_imerg_intersection = gpd.overlay(fire_daily, imerg_grid, how='intersection',keep_geom_type=False)
    fire_imerg_intersection['grid intersection area (ha)'] =fire_imerg_intersection['geometry'].area/10000
    fire_imerg_intersection['weights'] = fire_imerg_intersection['grid intersection area (ha)']/fire_imerg_intersection['fire area (ha)'] 
    
    inds_imerg_range = np.where(pd.to_datetime(fire_imerg_intersection['Current Day'].values)<=np.datetime64('2020-10-31'))
    
    fire_imerg_intersection = fire_imerg_intersection.iloc[inds_imerg_range]
    fire_imerg_intersection = fire_imerg_intersection.set_index(['Current Day', 'lat', 'lon'])
    fire_imerg_intersection_xr = fire_imerg_intersection.to_xarray()
    
    #load in rave data associated with the fire
    imerg_filenames = make_imerg_file_namelist(times)
    dat_imerg = xr.open_mfdataset(imerg_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_imerg = dat_imerg.assign_coords({'time':time}) #assign coords to make it clearer to work with
    
    print(fire_imerg_intersection_xr['lat'].values)
    #select the locations and times we want
    dat_imerg_sub = dat_imerg.sel(lat = fire_imerg_intersection_xr['lat'].values, 
                                lon = fire_imerg_intersection_xr['lon'].values,method='nearest',
                                time = pd.to_datetime(fire_imerg_intersection_xr['Current Day'].values))
        
    
    ndays = len(fire_imerg_intersection_xr['Current Day'])
    
    #preallocate space for the output
    df_imerg = pd.DataFrame({'day':np.zeros(ndays),'IMERG.FINAL.v6_DC':np.zeros(ndays),\
                          'IMERG.FINAL.v6_DMC':np.zeros(ndays),  'IMERG.FINAL.v6_FFMC':np.zeros(ndays),
                        'IMERG.FINAL.v6_ISI':np.zeros(ndays),'IMERG.FINAL.v6_BUI':np.zeros(ndays),
                            'IMERG.FINAL.v6_FWI':np.zeros(ndays),  'IMERG.FINAL.v6_DSR':np.zeros(ndays)})

    df_imerg['day'].iloc[:] = pd.to_datetime(fire_imerg_intersection_xr['Current Day'].values)
    for var in varis:
        df_imerg[var] = np.nansum(fire_imerg_intersection_xr['weights'].values*dat_imerg_sub[var].values, axis=(1,2))
    
    return df_imerg

In [ ]:
def make_imerg_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/FWI_IMERG/WESTUS_FWI.IMERG.FINAL.v6.Daily.Default.FULLDATE.nc'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
        
    return base_filename_list

## CPC FWI

In [ ]:
def cpc_fwi_timeseries(df, time):
    varis = ['CPC_DC','CPC_DMC','CPC_FFMC',
             'CPC_ISI','CPC_BUI','CPC_FWI',
             'CPC_DSR'] 

    #load in the grid
    cpc_grid = gpd.read_file('CPC_FWI_GRID.geojson')
    cpc_grid = cpc_grid.to_crs(epsg=3347)

    #do the intersection, not with a for loop!
    fire_cpc_intersection = gpd.overlay(fire_daily, cpc_grid, how='intersection',keep_geom_type=False)
    fire_cpc_intersection['grid intersection area (ha)'] =fire_cpc_intersection['geometry'].area/10000
    fire_cpc_intersection['weights'] = fire_cpc_intersection['grid intersection area (ha)']/fire_cpc_intersection['fire area (ha)'] 
    
    inds_cpc_range = np.where(pd.to_datetime(fire_cpc_intersection['Current Day'].values)<=np.datetime64('2020-10-31'))
    
    fire_cpc_intersection = fire_cpc_intersection.iloc[inds_cpc_range]
    fire_cpc_intersection = fire_cpc_intersection.set_index(['Current Day', 'lat', 'lon'])
    fire_cpc_intersection_xr = fire_cpc_intersection.to_xarray()
    
    #load in rave data associated with the fire
    cpc_filenames = make_cpc_file_namelist(time)
    dat_cpc = xr.open_mfdataset(cpc_filenames,concat_dim='time',combine='nested',compat='override', coords='all')
    dat_cpc = dat_cpc.assign_coords({'time':time}) #assign coords to make it clearer to work with
    
    print(fire_cpc_intersection_xr['lat'].values)
    #select the locations and times we want
    dat_cpc_sub = dat_cpc.sel(lat = fire_cpc_intersection_xr['lat'].values, 
                                lon = fire_cpc_intersection_xr['lon'].values,method='nearest',
                                time = pd.to_datetime(fire_cpc_intersection_xr['Current Day'].values))
        
    
    ndays = len(fire_cpc_intersection_xr['Current Day'])
    
    #preallocate space for the output
    df_cpc = pd.DataFrame({'day':np.zeros(ndays),'CPC_DC':np.zeros(ndays),\
                          'CPC_DMC':np.zeros(ndays),  'CPC_FFMC':np.zeros(ndays),
                        'CPC_ISI':np.zeros(ndays),'CPC_BUI':np.zeros(ndays),
                            'CPC_FWI':np.zeros(ndays),  'CPC_DSR':np.zeros(ndays)})

    df_cpc['day'].iloc[:] = pd.to_datetime(fire_cpc_intersection_xr['Current Day'].values)
    for var in varis:
        df_cpc[var] = np.nansum(fire_cpc_intersection_xr['weights'].values*dat_cpc_sub[var].values, axis=(1,2))
    
    return df_cpc

In [ ]:
def make_cpc_file_namelist(time):
    base_filename = '/data2/lthapa/YEAR/FWI_CPC/WESTUS_FWI.CPC.Daily.Default.FULLDATE.nc'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('YEAR',time[jj].strftime('%Y')).\
                                    replace('FULLDATE',time[jj].strftime('%Y%m%d'))
        
    return base_filename_list

## HRRR

In [ ]:
def hrrr_timeseries(df,day_start_hour):  
    df_hrrr = pd.DataFrame({'day': np.zeros(len(df)),'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'weasd':np.zeros(len(df)),'soilm':np.zeros(len(df)), 'esat_2m':np.zeros(len(df)),
                             'e_2m': np.zeros(len(df)),'vpd_2m':np.zeros(len(df)), 'hwp':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'veggie':np.zeros(len(df)), 'dewpt':np.zeros(len(df)),
                             'veg_term':np.zeros(len(df)),'gust_max_term':np.zeros(len(df)), 'dd_term':np.zeros(len(df)),
                             'mois_term': np.zeros(len(df)),'temp_2m':np.zeros(len(df)), 'q_2m':np.zeros(len(df)), 
                             'gust_sfc':np.zeros(len(df)),'snowc_term':np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    #load in the grid
    hrrr_grid = gpd.read_file('HRRR_GRID.geojson')
    hrrr_grid = hrrr_grid.to_crs(epsg=3347) #put into lambert conformal conic 
    
    #do the intersection, not with a for loop!
    for geom in dg['geometry']:
        print(geom)
    
    #fire_hrrr_intersection = gpd.overlay(df, hrrr_grid, how='intersection',keep_geom_type=False)
    
    
    """
    fire_hrrr_intersection['grid intersection area (ha)'] =fire_hrrr_intersection['geometry'].area/10000
    fire_hrrr_intersection['weights'] = fire_hrrr_intersection['grid intersection area (ha)']/fire_hrrr_intersection['fire area (ha)'] 
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values)
    times_utc = np.unique(fire_hrrr_intersection['UTC Day'].values)
    
    count = 0
    for today in times_intersect:
        #get the time
        df_sub = fire_hrrr_intersection.iloc[np.where(fire_hrrr_intersection[str(day_start_hour)+ 'Z Start Day'].values==today)]
        df_sub = df_sub.set_index([str(day_start_hour)+ 'Z Start Day', 'lat', 'lon'])
        intersection_sub = df_sub.to_xarray() #polygon and weights for today

        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today)+np.timedelta64(1,'D'))
        files_back = make_hrrr_file_namelist(times_back)
        print(files_back)
        
        #load in all the merra files associated with this lookback window
        dat_merra = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')
    
        #add the derived data (svp, vp, vpd)
        dat_merra=dat_merra.assign(ESAT=sat_vap_press(dat_merra.TLML))
        dat_merra=dat_merra.assign(E=vap_press(dat_merra.QLML, dat_merra.TLML))
        dat_merra=dat_merra.assign(VPD=dat_merra.ESAT-dat_merra.E)
        
        merra_daily_mean = dat_merra.resample(time='24H',base=day_start_hour, label='left').mean(dim='time') #take the daily mean        
        merra_daily_mean_region = merra_daily_mean.sel(lat = np.unique(intersection_sub['lat'].values),
                                  lon = np.unique(intersection_sub['lon'].values)) #get the location of the overlaps
        
        hd0 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((merra_daily_mean_region['SPEEDLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        t = np.nansum((merra_daily_mean_region['TLML'].sel(time=np.datetime64(today+ ' '+str(day_start_hour)+':00:00')).values)*(intersection_sub['weights'].values))
        
        df_merra.iloc[count,:] = [today+ ' '+str(day_start_hour)+':00:00',t,hd0,w,hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        dat_merra.close()
        count =count+1
    return df_merra
    """

In [ ]:
def make_hrrr_file_namelist(time):
    base_filename = '/data2/lthapa/ML_py/pygraf/Processed_HRRR_FULLDATE.nc4'
    base_filename_list = np.repeat(base_filename, len(time))

    
    for jj in range(len(time)):
        base_filename_list[jj] = base_filename_list[jj].replace('FULLDATE',time[jj].strftime('%Y%m%d%H'))
    return base_filename_list

## Testing

In [ ]:
print(times)
names = make_merra_file_namelist(times)
print(names)
dat_cpc = xr.open_mfdataset(names,concat_dim='time',combine='nested',compat='override', coords='all')
print(dat_cpc)

In [ ]:
fire_daily = gpd.read_file('./fire_polygons/lake_VIIRS_daily.geojson') #polygons and attributes
merra_grid = gpd.read_file('MERRA_GRID.geojson')

In [ ]:
fire_total = fire_daily.dissolve()
print(fire_total)

In [ ]:
fire_total = fire_total.to_crs(epsg=4326) #put into wsg84

fire_total['geometry'].iloc[0].bounds

In [ ]:
def bbox2ij(lon,lat,bbox):
    bbox=np.array(bbox)
    mypath=np.array([bbox[[0,1,1,0]],bbox[[2,2,3,3]]]).T
    p = path.Path(mypath)
    points = np.vstack((lon.flatten(),lat.flatten())).T
    n,m = np.shape(lon)
    inside = p.contains_points(points).reshape((n,m))
    ii,jj = np.meshgrid(range(m),range(n)) #ii is the columns, jj is the rows
    return min(ii[inside]),max(ii[inside]),min(jj[inside]),max(jj[inside])

In [ ]:
# hdw_type= 'max' (use/report daily maxes), 'ave' (use/report averages)
def merra_timeseries_old(df, time, hdw_type):
    
    df_merra = pd.DataFrame({'day': np.zeros(len(df)),'temp':np.zeros(len(df)), 'vpd':np.zeros(len(df)), 
                             'wind':np.zeros(len(df)),'hd0w0':np.zeros(len(df)), 'hd1w0':np.zeros(len(df)),
                             'hd2w0':np.zeros(len(df)),'hd3w0':np.zeros(len(df)), 'hd4w0':np.zeros(len(df)),
                             'hd5w0':np.zeros(len(df))})
    #load in the grid
    merra_grid = gpd.read_file('MERRA_GRID.geojson')
    merra_grid = merra_grid.to_crs(epsg=3347) #put into lambert conformal conic 
    
    #do the intersection, not with a for loop!
    fire_merra_intersection = gpd.overlay(df, merra_grid, how='intersection',keep_geom_type=False)
    fire_merra_intersection['grid intersection area (ha)'] =fire_merra_intersection['geometry'].area/10000
    fire_merra_intersection['weights'] = fire_merra_intersection['grid intersection area (ha)']/fire_merra_intersection['fire area (ha)'] 
    
    
    #loop over all of the days we have intersections
    times_intersect = np.unique(fire_merra_intersection['Current Day'].values)
    
    count = 0
    for today in times_intersect:        
        #get the time
        df_sub = fire_merra_intersection.iloc[np.where(fire_merra_intersection['Current Day'].values==today)]
        df_sub = df_sub.set_index(['Current Day', 'lat', 'lon'])
        intersection_sub = df_sub.to_xarray() #polygon and weights for today

        times_back = pd.date_range(start=np.datetime64(today)-np.timedelta64(5,'D'), end=np.datetime64(today))
        files_back = make_merra_file_namelist(times_back)

        #load in all the merra files associated with this lookback window
        dat_merra = xr.open_mfdataset(files_back,concat_dim='time',combine='nested',compat='override', coords='all')

        #add the derived data (svp, vp, vpd)
        dat_merra=dat_merra.assign(ESAT=sat_vap_press(dat_merra.TLML))
        dat_merra=dat_merra.assign(E=vap_press(dat_merra.QLML, dat_merra.TLML))
        dat_merra=dat_merra.assign(VPD=dat_merra.ESAT-dat_merra.E)
    
        merra_daily_mean = dat_merra.resample(time='D').mean(dim='time') #take the daily mean
        
        merra_daily_mean_region = merra_daily_mean.sel(lat = np.unique(intersection_sub['lat'].values),
                                  lon = np.unique(intersection_sub['lon'].values)) #get the location of the overlaps

        hd0 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today)).values)*(intersection_sub['weights'].values))
        hd1 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today)-np.timedelta64(1,'D')).values)*
                     (intersection_sub['weights'].values))
        hd2 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today)-np.timedelta64(2,'D')).values)*
                     (intersection_sub['weights'].values))
        hd3 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today)-np.timedelta64(3,'D')).values)*
                     (intersection_sub['weights'].values))
        hd4 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today)-np.timedelta64(4,'D')).values)*
                     (intersection_sub['weights'].values))
        hd5 = np.nansum((merra_daily_mean_region['VPD'].sel(time=np.datetime64(today)-np.timedelta64(5,'D')).values)*
                     (intersection_sub['weights'].values))
        w = np.nansum((merra_daily_mean_region['SPEEDLML'].sel(time=np.datetime64(today)).values)*(intersection_sub['weights'].values))
        t = np.nansum((merra_daily_mean_region['TLML'].sel(time=np.datetime64(today)).values)*(intersection_sub['weights'].values))
        
        df_merra.iloc[count,:] = [today,t,hd0,w,hd0*w,hd1*w,hd2*w,hd3*w,hd4*w,hd5*w]
        dat_merra.close()
        count =count+1
    return df_merra
    